## Following steps to complete this are: 
1. Read in data (questions and answers) 
2. Preprocess data, remove stop words, tokenize ect. 
3. Train word2vec model; create list of question sentences and response sentences 
4. Preprocess user input, vectorize user input, calculate cosine distance (measures the angle between two vectors as a measure of similarity)... we could also use euclidean distance and compare which is better. 
5. Handle situation if the topic is not in the topic trained but asking user to rephrase?
6. Sort similarity and return top questions and score 


In [1]:
import re
import os, sys
import string
import pandas as pd 
import spacy 
import numpy as np
from nltk import *
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

import random



In [2]:
df[df.CR_response == 1].message
responses = df[df.CR_response == 1].message
df_responses = pd.Series(responses).to_frame()

df_responses.to_csv('responses_dirty.txt')

In [3]:
pd.set_option('display.max_colwidth', 100)

q = df[df.parent_id.isna()].message

question_df = pd.DataFrame(q)
question_df
question_df.to_csv('questions_dirty.txt')
feedback_string=question_df.message.to_string()
sentences = tokenize.sent_tokenize(feedback_string)
filtered_sent = [i for i in sentences if '?' in i]
filtered_sent[5]

"We all...\n124      I was trying to set up my account per my landlord's request and the system has locked me out and...\n125      Hi, how do I disable receiving an email notification when a TC message is received?"

In [4]:
mystring =  feedback_string 
keyword = '?'
befor_keyowrd, keyword, after_keyword = mystring.partition(keyword)

In [ ]:
f = open('dwolla_temp.csv','r', errors = 'ignore')
raw = f.read()
raw = raw.lower()

sent_tokens = sent_tokenize(raw) # converts to list of sentences 
word_tokens = word_tokenize(raw) # converts to list of words


lemmer = stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(word_tokenize(text.lower().translate(remove_punct_dict)))


GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["Good day, How may I help you?", "Hello, How can i help?", "hello", "I am glad! You are talking to me."]



 # Checking for greetings
def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


# Generating response
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response


flag = True
print("DWOLLA ROO: Hi! My name is dwolla roo! I will do my best to assist in any queries you might have about dwolla") 
while(flag == True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!= 'bye'):
        if(user_response =='thanks' or user_response =='thank you' ):
            flag=False
            print("DWOLLA ROO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("DWOLLA ROO: "+greeting(user_response))
            else:
                print("DWOLLA ROO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("DWOLLA ROO: Bye! take care..")    
        
 

DWOLLA ROO: Hi! My name is dwolla roo! I will do my best to assist in any queries you might have about dwolla
hi
DWOLLA ROO: Hello, How can i help?
withdraw
DWOLLA ROO: click on the automatic withdraw toggle button:here you can enter a start date of withdraw, frequency and the destination account.
